In [ ]:
!pip install pororo pickle5

In [ ]:
from pororo import Pororo
from bs4 import BeautifulSoup as soup
import pandas as pd
import numpy as np
import re
import sys, os
import pickle

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
cd 

In [ ]:
def make_y_label(model, doc:str):
    return model(doc)

def remove_punctuation_mark(paragrahp:str):
    paragraph = re.sub(r'\([^)]*\)', " ", paragrahp)
    paragrahp = re.sub(r"&+#+[0-9]*?;", "", paragrahp)
    patten = r"[^ .,·?!:\'\”%/()A-Za-z0-9가-힣+]"
    paragrahp = re.sub(patten, " ", paragrahp)
    paragrahp = " ".join(paragrahp.split())
    return paragrahp


def remove_foreword(paragraph:str):
    try:
        paragraph = re.sub("(\\[[\\D]+] )", " ", paragraph)
    except:
        pass
    finally:
        return paragraph


def remove_reporter_name(paragraph:str):
    try:
        paragraph = re.sub("([가-힣]+ ?기자=)", " ", paragraph)
    except:
        pass
    finally:
        return paragraph


def remove_source_of_photos(paragraph:str):
    try:
        paragraph = re.sub("(사진=[\w가-힣 ?]+)", " ", paragraph)
    except:
        pass
    finally:
        return paragraph

def remove_phone_num(paragraph:str):
    paragraph = re.sub(
        "[0-9]{2,3}[-=+,#/\?:^$.@*\"~&%ㆍ!_\\‘|\(\)\[\]\<\>`'…》][0-9]{3,4}[-=+,#/\?:^$.@*\"※~&%ㆍ!_\\‘|\(\)\[\]\<\>`'…》][0-9]{3,4}", " ", paragraph)
    return paragraph

def sentence_split(paragrahp:str) -> list:
    return split_sentences(paragrahp, num_workers = 1)


def remove_html_tag(paragrahp:str):
    if paragrahp != paragrahp:
        paragrahp = "   "
    
    paragrahp = re.sub('<.*?>', '', paragrahp)
    remove_p_tag = [str(_.text) for _ in soup(paragrahp, features="lxml").find_all('p') if len(_.text) > 0]
    paragrahp = " ".join(remove_p_tag)
    return paragrahp

def remove_url_email(paragrahp:str):
  paragrahp = re.sub("((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*", " ", paragrahp)
  paragrahp = re.sub("'^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'", "", paragrahp)
  return paragrahp

def preprocess(model, docs:list, titles:list, num:int):
    doc_len = len(docs)//(25)
    l_idx = doc_len*(num-1)
    r_idx = doc_len*num
    
    if num == 25:
      r_idx = None

    docs = docs[l_idx:r_idx]
    titles = titles[l_idx:r_idx]
    

    result = list()
    for idx, doc in enumerate(docs):
      try:
        content = str(titles[idx])+str(doc)
        content = remove_url_email(content)
        content = remove_phone_num(content)
        content = remove_html_tag(content)
        content = remove_foreword(content)
        content = remove_reporter_name(content)
        content = remove_source_of_photos(content)
        content = remove_punctuation_mark(content)
        
        y_label = make_y_label(model, content)
        result.append((content, y_label))

        if idx%100==0:
          print(idx, "/////", len(docs), sep="")
      except:
        print(titles[idx])
        print(doc)
        break
    
    return result

In [ ]:
data_path=
df = pd.read_csv(data_path, engine='python', encoding='utf-8')
df = df.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)

docs = df.CONTENT.tolist()
titles = df.TITLE.tolist()

pororo_model = Pororo(task="summarization", model="abstractive", lang="ko")

In [ ]:
os.makedirs("./data", exist_ok=True)

for i in range(1, 26):
  num = i
  if num < 10:
    str_num = "0"+str(num)
  else:
    str_num = str(num)
  output_file_name = './data/data_'+str_num+'.pickle'
  if os.path.isfile(output_file_name):
    continue
  preprocessing_result = preprocess(pororo_model, docs, titles, num)
  
  with open(output_file_name, 'wb') as f:
      pickle.dump(preprocessing_result, f, pickle.HIGHEST_PROTOCOL)
  print("process end:", num)

In [ ]:
import pickle5 as pickle

total_list = list()
file_list = sorted(os.listdir("./data/"))
for file in file_list:
  print(file)
  with open("./data/"+file, 'rb') as f:
    data = pickle.load(f)
    total_list.extend(data)

with open('./data/data.pickle', 'wb') as f:
    pickle.dump(total_list, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle5 as pickle
with open('./data/data.pickle', 'rb') as f:
    data = pickle.load(f)

len(data)